# Activity 2:
## Gaussian Elimination and LU Factorizations

In [1]:
import numpy as np
from numpy import random as RA

In this activity, we will work to build an LU factorization algorithm from the ground up using only the most basic of built-in numpy commands. First, we'll review some array manipulation.

In [3]:
A=np.array([
    [1,2,4,-3],
    [2,6,1,7],
    [-0.5,9,3,3],
    [-1,2,-4,3]],"float64")

You can change the elements of an array individually like so:

In [4]:
B=A.copy()
print("B=\n",B)
B[0][0]=-99
B[2][3]=99
print("B=\n",B)

B=
 [[ 1.   2.   4.  -3. ]
 [ 2.   6.   1.   7. ]
 [-0.5  9.   3.   3. ]
 [-1.   2.  -4.   3. ]]
B=
 [[-99.    2.    4.   -3. ]
 [  2.    6.    1.    7. ]
 [ -0.5   9.    3.   99. ]
 [ -1.    2.   -4.    3. ]]


(recall the python indexing conventions: the top row/leftmost column are row/column zero)

Arithmetic operations performed on arrays (or rows of arrays) are performed element-wise. For example:

In [5]:
B=A.copy()
B[3]=4*A[2] #change the fourth row of B to be four times the third row of A
print("A=\n", A,"\nB=\n",B) #display A and B readably
C=np.identity(4)-3*A #construct a new matrix given as the difference between the identity matrix and 3 times A
print("C=\n",C) #display C readably


A=
 [[ 1.   2.   4.  -3. ]
 [ 2.   6.   1.   7. ]
 [-0.5  9.   3.   3. ]
 [-1.   2.  -4.   3. ]] 
B=
 [[ 1.   2.   4.  -3. ]
 [ 2.   6.   1.   7. ]
 [-0.5  9.   3.   3. ]
 [-2.  36.  12.  12. ]]
C=
 [[ -2.   -6.  -12.    9. ]
 [ -6.  -17.   -3.  -21. ]
 [  1.5 -27.   -8.   -9. ]
 [  3.   -6.   12.   -8. ]]


You can also reference rows or elements of the same array.

In [6]:
B=A.copy()
print("B=\n",B)
B[0]=3*B[1]
print("B=\n",B)
B[1]=B[0][3]*B[1]
print("B=\n",B)

B=
 [[ 1.   2.   4.  -3. ]
 [ 2.   6.   1.   7. ]
 [-0.5  9.   3.   3. ]
 [-1.   2.  -4.   3. ]]
B=
 [[ 6.  18.   3.  21. ]
 [ 2.   6.   1.   7. ]
 [-0.5  9.   3.   3. ]
 [-1.   2.  -4.   3. ]]
B=
 [[  6.   18.    3.   21. ]
 [ 42.  126.   21.  147. ]
 [ -0.5   9.    3.    3. ]
 [ -1.    2.   -4.    3. ]]


You can also swap rows of an array by calling a list of indices.

In [7]:
B=A.copy()
print("B=\n",B)
B[[0,3]]=B[[3,0]] #read this as "set the 0th row and the 3rd row of B to be the 3rd and 0th rows of B"
print("B=\n",B)
B[[1,2,3]]=B[[3,1,2]]
print("B=\n",B)

B=
 [[ 1.   2.   4.  -3. ]
 [ 2.   6.   1.   7. ]
 [-0.5  9.   3.   3. ]
 [-1.   2.  -4.   3. ]]
B=
 [[-1.   2.  -4.   3. ]
 [ 2.   6.   1.   7. ]
 [-0.5  9.   3.   3. ]
 [ 1.   2.   4.  -3. ]]
B=
 [[-1.   2.  -4.   3. ]
 [ 1.   2.   4.  -3. ]
 [ 2.   6.   1.   7. ]
 [-0.5  9.   3.   3. ]]


**You try:** copy $A$ (from above) to a new array $B$. move the bottom row to the top and shift the other rows down. Then, negate the new bottom row and multiply the new top row by $2$. Then, let $C=B-A$.

In [19]:
# YOUR CODE HERE
B=A.copy()
B[[3,0]] = B[[0,3]]
B[[3,2]] = B[[2,3]]
B[[2,1]] = B[[1,2]]
B[3] = B[3] * (-1)
B[0] = 2*B[0]

C = np.array([[B[0] - A[0]], [B[1] - A[1]],[B[2] - A[2]],[B[3] - A[3]]])
print("A=\n",A,"\nB=\n",B,"\nC=\n",C)
#desired output:
#A=
# [[ 1.   2.   4.  -3. ]
# [ 2.   6.   1.   7. ]
# [-0.5  9.   3.   3. ]
# [-1.   2.  -4.   3. ]]
#B=
# [[-2.   4.  -8.   6. ]
# [ 1.   2.   4.  -3. ]
# [ 2.   6.   1.   7. ]
# [ 0.5 -9.  -3.  -3. ]]
#C=
# [[ -3.    2.  -12.    9. ]
# [ -1.   -4.    3.  -10. ]
# [  2.5  -3.   -2.    4. ]
# [  1.5 -11.    1.   -6. ]]


A=
 [[ 1.   2.   4.  -3. ]
 [ 2.   6.   1.   7. ]
 [-0.5  9.   3.   3. ]
 [-1.   2.  -4.   3. ]] 
B=
 [[-2.   4.  -8.   6. ]
 [ 1.   2.   4.  -3. ]
 [ 2.   6.   1.   7. ]
 [ 0.5 -9.  -3.  -3. ]] 
C=
 [[[ -3.    2.  -12.    9. ]]

 [[ -1.   -4.    3.  -10. ]]

 [[  2.5  -3.   -2.    4. ]]

 [[  1.5 -11.    1.   -6. ]]]


### Exercise 1: Regular Gaussian Elimination
First, let's build an function to perform Gaussian Elimination in the case of a regular matrix.
You are free to build your algorithm as you see fit (without using built-in functions which do the hard part for you), but here is a basic outline of one:
* Set a variable n for the size of `A` using the `shape` method. `A.shape` returns a tuple with the dimensions of `A`, so if `A` is $n\times n$, `A.shape[0]` will return `n`.
* Copy A to a new array `U` (we'll generally avoid altering the input of our functions when unnecessary)
* Use a for loop to iterate over `i` in `range(n)`. This parameter will represent the pivot we are currently working with. Then, within the for loop:
    * "Clear out" column i below the pivot `U[i][i]` by iterating with a new for loop over `j` in `range(i+1,n)`. This `j` will parameterize the row we are currently "clearing out."
        * Change the `j`th row `U[j]` by subtracting the appropriate multiple of the `i`th row `U[i]` to clear out the $(i,j)$th entry `U[i][j]`.

For the moment, we'll assume that the input is regular--don't worry for now about what happens in the case it is not.

In [31]:
def my_elim(A):
# YOUR CODE HERE
  n = A.shape[0]
  U = A.copy()

  for i in range(n):
    for j in range(i+1,n):
      pivot_factor = U[j,i]/U[i,i]
      U[j,i:] = ((-1*pivot_factor) * (U[i,i:])) + (U[j,i:])

  return U

Try out your elimination function on the test matrix below (you have our word that it's regular).

In [32]:
A=np.array([[5., 4., 0.],
        [6., 5., 4.],
        [0., 6., 5.]])
my_elim(A)
#desired output:
#        array([[   5. ,    4. ,    0. ],
#               [   0. ,    0.2,    4. ],
#               [   0. ,    0. , -115. ]])

array([[   5. ,    4. ,    0. ],
       [   0. ,    0.2,    4. ],
       [   0. ,    0. , -115. ]])

### Exercise 2: LU Factorization
Now, we'll add in code to compute the LU factorization of `A`.
First, we'll need code to construct elementary matrices.
In the cell below, define a function to give the $n\times n$ elementary matrix $E$ where $E_{ij}=a$.

One helpful numpy built-in function is `np.identity`. Try it out in the next cell

In [33]:
np.identity(5)

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

**Exercise: (a)** Now, use that to write your `elem_matrix` function. Now, use that to write your `elem_matrix` function.

___

## Exercise 2: LU Factorization
Now, we'll add in code to compute the LU factorization of `A`.
First, we'll need code to construct elementary matrices.

**(a)** Above, we saw how to perform elementary row operations on numpy `ndarray`s.
Now, let's formalize that as a function.

**(i): Write a function** which takes as input a matrix `A`, a constant `a` and indices `i` and `j` and adds `a` times row `i` to row `j`.

In [36]:
def row_op(A,a,i,j):
    #YOUR CODE HERE
    B = A.copy()

    B[j] = (B[i])*a + B[j]

    return B

In [37]:
#Testing:
M=np.array([[(i+2)*(i+j+1) for j in range(4)] for i in range(4)], "float64")
row_op(M,-10,0,3)
#desired output:
#array([[  2.,   4.,   6.,   8.],
#       [  6.,   9.,  12.,  15.],
#       [ 12.,  16.,  20.,  24.],
#       [  0., -15., -30., -45.]])

array([[  2.,   4.,   6.,   8.],
       [  6.,   9.,  12.,  15.],
       [ 12.,  16.,  20.,  24.],
       [  0., -15., -30., -45.]])

___

Now, try another approach.
The $n\times n$ identity matrix is given by the numpy function `np.identity(n)`.

**(ii): Write a function** which performs the row operation above on  the identity matrix $I$ to yield the matrix $E$, then multiplies $E$ and $A$ to yield $EA$.
Compare this new function and `row_op`.
How is their output similar? How is it different?

<details>
    <summary>
        <b> Hint:</b>
        (Click here to open)
    </summary>
    
Use `A.shape` to determine which size your identity matrix should be!
    
</details>

In [51]:
def row_op2(A,a,i,j):
    #YOUR CODE HERE
    n = A.shape[0]
    E = np.identity(n)

    E[j,i] = a

    return np.dot(E, A)


In [52]:
#Testing:
row_op2(np.identity(4),-10,0,3)
#desired output:
#array([[  1.,   0.,   0.,   0.],
#       [  0.,   1.,   0.,   0.],
#       [  0.,   0.,   1.,   0.],
#       [-10.,   0.,   0.,   1.]])

array([[  1.,   0.,   0.,   0.],
       [  0.,   1.,   0.,   0.],
       [  0.,   0.,   1.,   0.],
       [-10.,   0.,   0.,   1.]])

In [ ]:
#Use this cell to experiment with row_op and row_op2

___

The matrix $E$ we constructed is the *Elementary matrix of type 1* representing the row operation!

In the cell below, **(iii): define a function** to give the $n\times n$ elementary matrix $E$ where $E_{ij}=a$.


In [50]:
def elem_matrix(n,a,i,j):
# YOUR CODE HERE
    E = np.identity(n)

    E[j,i] = a
    return E

___

**(b)** Now let's modify your `my_elim` function to produce an LU factorization of your input matrix `A`.

Some things to keep in mind:
* You still don't need to worry about the case that `A` is irregular--for now, assume it is regular.
* The `np.dot` function performs matrix multiplication (that is, $AB$ is given by `np.dot(A,B)`).
* Page 17 of Olver-Shakiban gives the formula you'll need to form `L`. Initialize it as the appropriately-sized identity matrix, and multiply by the appropriate elementary matrix in each step of the iteration

In [62]:
def my_LU(A):
#YOUR CODE HERE
  n = A.shape[0]
  U = A.copy()
  L = np.identity(n)
  for i in range(n):
    for j in range(i+1,n):
      pivot_factor = U[j,i]/U[i,i]
      U[j,i:] = ((-1*pivot_factor) * (U[i,i:])) + (U[j,i:])
      L[j,i] = pivot_factor
  return (L,U)

Try it out in the next two cells.

In [63]:
#testing
A=np.array([[5., 4., 0.],
        [6., 5., 4.],
        [0., 6., 5.]])
(L,U)=my_LU(A)
print("L=\n",L,"L=\n",U)
#desired output:
#    (array([[ 1. ,  0. ,  0. ],
#            [ 1.2,  1. ,  0. ],
#            [ 0. , 30. ,  1. ]]),
#     array([[   5. ,    4. ,    0. ],
#            [   0. ,    0.2,    4. ],
#            [   0. ,    0. , -115. ]]))

L=
 [[ 1.   0.   0. ]
 [ 1.2  1.   0. ]
 [ 0.  30.   1. ]] L=
 [[   5.     4.     0. ]
 [   0.     0.2    4. ]
 [   0.     0.  -115. ]]


In [64]:
#Check if reconstruction works!
print("A=\n",A,"\nLU=\n",np.dot(L,U))

A=
 [[5. 4. 0.]
 [6. 5. 4.]
 [0. 6. 5.]] 
LU=
 [[5. 4. 0.]
 [6. 5. 4.]
 [0. 6. 5.]]


___

**(c):**
So far, we haven't had to worry about the case of irregular input. However, in practice, your code needs to at least let the user know when they have entered as input an irregular matrix.
Python performs error handling by using the built-in `raise` command to raise an `Exception`, which is a message given to the user to inform them what's gone wrong. Below is a quick demonstration:

In [66]:
def even_division(n):
    if n%2 == 1: #check if n odd
        raise Exception("input integer is odd")
    return n//2 #divide by 2


In [72]:
even_division(4)

2

In [73]:
even_division(5)

Exception: input integer is odd

**Exercise:**
In the below cell, modify your code for `my_LU` to raise an exception when the input is an irregular matrix. The `Exception` should say "Input matrix irregular."

In [75]:
def my_LU_safe(A):
#YOUR CODE HERE
  n = A.shape[0]
  U = A.copy()
  L = np.identity(n)


  for i in range(n):
    if A[i,i] == 0:
      raise Exception ("Input matrix irregular.")
    for j in range(i+1,n):
      pivot_factor = U[j,i]/U[i,i]
      U[j,i:] = ((-1*pivot_factor) * (U[i,i:])) + (U[j,i:])
      L[j,i] = pivot_factor
  return (L,U)

In [76]:
#Testing
A=np.array([[i*j for j in range(5)] for i in range(5)]) #This should throw an exception
my_LU_safe(A)

Exception: Input matrix irregular.

___

### Exercise 3: Solving Linear Systems
One of the major use cases for LU factorization is solving systems by forward- and back- substitution, as described in Olver-Shakiban, section 1.3.
To solve a system $A\vec{x}=\vec{b}$, this works by
* computing an LU factorization $A=LU$,
* Solving the system $L\vec{c}=\vec{b}$
* Solving the system $U \vec{x}= \vec{c}$

**Exercise:** Below, give code which implements this using `my_LU`. Do *not* use any `numpy` built-in functions which trivialize any part of this (such as taking matrix inverses)--rather you should write code to perform the substitution manually. You may choose to proceed by writing a function for back substitution then another for forward substitution if you wish--that approach is outlined below.
Be sure to test your code before you submit!

In [84]:
def forward_sub(L,b):
  n = L.shape[0]
  c = np.zeros(n, dtype=float)

  for i in range(n):
      s = 0.0
      for k in range(i):
          s += L[i, k] * c[k]
      c[i] = b[i] - s
  return c

def back_sub(U, c):
    n = U.shape[0]
    x = np.zeros(n, dtype=float)
    for i in range(n-1, -1, -1):
        s = 0.0
        for k in range(i+1, n):
            s += U[i, k] * x[k]
        if U[i, i] == 0:
            raise ZeroDivisionError("Zero pivot in U.")
        x[i] = (c[i] - s) / U[i, i]
    return x


In [87]:
def my_solver(A,b):
  (L,U) = my_LU_safe(A)
  c = forward_sub(L,b)

  x = back_sub(U, c)

  return x

In [107]:
#testing your code
A=RA.rand(10,10)
b=RA.rand(10)
x=my_solver(A,b)
Ax=np.dot(A,x)
print(Ax-b) #should get vector of zeros, or close to it.
max(Ax-b)<10**(-14) #check that result is within numerical errors of correct

[ 0.00000000e+00 -3.33066907e-16 -2.22044605e-16 -2.22044605e-16
 -3.33066907e-16 -2.22044605e-16  1.11022302e-16 -4.44089210e-16
  1.11022302e-16  4.44089210e-16]


np.True_

___

### Exercise 4 (Challenge!): Hilbert Matrices and Numerical Instability.
LU sadly suffers from some numerical instability.
This rears its head in particular when the input matrix $A$ is *ill-conditioned*, meaning that the product $A\vec x$ can change significantly with relatively small perturbations of the input.
A classic example is the $n$th *Hilbert matrix* $H_n$, given entrywise by $$H_n=\left(\frac{1}{i+j-1}\right)_{i,j}$$ where $i,j$ range over $1,\dots,n$.
For example, the third Hilbert matrix is
$$H_3=\begin{pmatrix}
1 & \frac12 & \frac13\\
\frac12 & \frac13 & \frac14\\
\frac13 & \frac14 & \frac15
\end{pmatrix}.$$

**Exercise: (a)** Write a function `hilbert` which takes as input an integer `n` and returns the `n`th Hilbert matrix $H_n$. Be careful to account for python's indexing.

In [117]:
def hilbert(n):
  H = np.empty((n,n),dtype=float)
  for i in range(n):
    for j in range(n):
      H[i,j] = 1/((i+1)+(j+1)-1)
  return H

In [118]:
hilbert(5)

array([[1.        , 0.5       , 0.33333333, 0.25      , 0.2       ],
       [0.5       , 0.33333333, 0.25      , 0.2       , 0.16666667],
       [0.33333333, 0.25      , 0.2       , 0.16666667, 0.14285714],
       [0.25      , 0.2       , 0.16666667, 0.14285714, 0.125     ],
       [0.2       , 0.16666667, 0.14285714, 0.125     , 0.11111111]])

___

**(b):** A good way to check the numerical accuracy of a linear system solver such as `my_solver` for a given matrix $A$ is to generate a random vector $\vec x_0$ of appropriate dimension, set $\vec b$ to be the product $\vec b = A \vec x_0$, and then set $\vec x_1$ to be the result of running the solver on the linear system $A\vec x= \vec b$.
In principle, $\vec x_1$ should be equal to $\vec x_0$, but as we'll see, thanks to rounding errors, this is not always the case in practice.
Write a function `hilbert_checker` takes as input a parameter `max_discrepancy` and then loops to perform the process above for increasing dimensions $n$ starting with $n=1$. Use `np.random.rand` (imported for convenience as `RA.rand`) to generate your random vectors. At each stage in the loop, print the current entry `n` and the maximum entry of the absolute difference $\left \lvert \vec x_0- \vec x_1\right \rvert$. Your loop should terminate when the maximum difference in absolute value between $\vec x_0$ and $\vec x_1$ exceeds `max_discrepancy`, returning the dimension `n` at which that occurred. How long does it take for `max_discrepancy` to exceed $10^{-8}$? What about $10^{-4}$,  $10^{-2}$,  1, 100, or 10000? Try running `hilbert_checker` a few times for the same value of `max_discrepancy`. Do you get similar results? Write a few words below summarizing your observations.

*Note:* While the python built-in function `abs` does work as expected with `numpy` arrays, the same cannot be said for `max`. Rather, use `np.max`.

In [125]:
def hilbert_checker(max_discrepancy=1):
  n = 0
  while True:
        n += 1
        A  = hilbert(n).astype(float)
        x0 = RA.rand(n)
        b  = np.dot(A,x0)
        x1 = my_solver(A, b)

        max = np.abs(x0 - x1).max()
        print(f"n={n:3d}  max|x0 - x1| = {max:.3e}")

        if dmax > max_discrepancy:
            return n

In [123]:
#Use this cell to test it out!
hilbert_checker(1e-8)
hilbert_checker(1e-4)
hilbert_checker(1e-2)

n=  1  max|x0 - x1| = 0.000e+00
n=  2  max|x0 - x1| = 2.220e-16
n=  3  max|x0 - x1| = 1.199e-14
n=  4  max|x0 - x1| = 2.958e-13
n=  5  max|x0 - x1| = 4.567e-12
n=  6  max|x0 - x1| = 9.472e-11
n=  7  max|x0 - x1| = 5.480e-09
n=  8  max|x0 - x1| = 7.142e-08
n=  1  max|x0 - x1| = 0.000e+00
n=  2  max|x0 - x1| = 2.220e-16
n=  3  max|x0 - x1| = 8.687e-15
n=  4  max|x0 - x1| = 7.316e-14
n=  5  max|x0 - x1| = 2.541e-12
n=  6  max|x0 - x1| = 1.185e-10
n=  7  max|x0 - x1| = 1.451e-08
n=  8  max|x0 - x1| = 2.050e-07
n=  9  max|x0 - x1| = 1.367e-05
n= 10  max|x0 - x1| = 2.714e-04
n=  1  max|x0 - x1| = 0.000e+00
n=  2  max|x0 - x1| = 5.551e-17
n=  3  max|x0 - x1| = 5.274e-15
n=  4  max|x0 - x1| = 8.138e-14
n=  5  max|x0 - x1| = 1.152e-11
n=  6  max|x0 - x1| = 2.105e-10
n=  7  max|x0 - x1| = 9.635e-09
n=  8  max|x0 - x1| = 2.842e-07
n=  9  max|x0 - x1| = 1.636e-06
n= 10  max|x0 - x1| = 3.255e-04
n= 11  max|x0 - x1| = 9.188e-03
n= 12  max|x0 - x1| = 4.203e-01


12

(This is a blank markdown cell to write down some observations)